# RAG

## Requirements

In [1]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown

## Dataset

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=97daa431-40ef-46eb-8d20-c8550ae5ca26
To: /content/IMDB_crawled.json
100% 292M/292M [00:08<00:00, 32.9MB/s]


## Config

In [3]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [4]:
import pandas as pd

df = pd.read_json('IMDB_crawled.json')

In [5]:
import os

os.makedirs('data', exist_ok=True)

# preprocess your data and only store the needed data as the context window for embedding model is limited
df= df[['id', 'title', 'genres', 'first_page_summary']]

df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [9]:
import pickle

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

# load the csv
docs= CSVLoader('data/imdb.csv').load()

# load the embeddings model
emb_model= HuggingFaceEmbeddings(model_name= Config.EMBEDDING_MODEL_NAME, encode_kwargs={"normalize_embeddings": True})

# save embed the documents using the model in a vectorstore
vectorstore= FAISS.from_documents(docs, emb_model)

with open("data/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

load the vectorstore as a retriever.

In [10]:
with open("data/vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever= vectorstore.as_retriever(top_k= 5)

## LLM

load the quantized LLM.

In [11]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# load the quantization config
bnb_config= BitsAndBytesConfig(load_in_8bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")

model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM= pipeline("text-generation", model= model, tokenizer= tokenizer, max_length=50000)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY: {text}")
        return text

query_transform_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""<|system|>You are a helpful assistant.
{messages}
<|user|>
give me the search query about the above conversation.
<|assistant|>"""
)

# init the query chain
query_transforming_retriever_chain= LLMChain(llm= llm, prompt= query_transform_prompt, output_parser= LoggerStrOutputParser())


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    input_variables=["context", "messages"],
    template="""<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

{context}
-----------------
{messages}
<|assistant|>""")

# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/
# init the retriver chain
retrieval_chain= ({"context": retriever, "messages": RunnablePassthrough()}|prompt|llm|StrOutputParser())



write the conversation helper class for easier testing.

In [15]:
class Conversation:
    def __init__(self):
        self.messages = []

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        # concatenate the messages with the roles in the instruction format
        concated_messages= []
        for r, m in self.messages:
            concated_messages.append(f"{r}: {m}")
        return ("\n".join(concated_messages))

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        # invoke the chain
        response= retrieval_chain.invoke(messages)
        self.add_assistant_message(response)
        return response

## Test

talk with the RAG to see how good it performs.

In [16]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print(A)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

[Document(page_content="id: tt27555922\ntitle: Goodfellas 2\ngenres: ['Short', 'Comedy']\nfirst_page_summary: Two gangsters meet and fall madly in love, and have a little bit of gay romance but their Italian brothers won't let that one slide so they all hit the gritty hop on the Fortnite and win som... Read all", metadata={'source': 'data/imdb.csv', 'row': 3951}), Document(page_content="id: tt0765429\ntitle: American Gangster\ngenres: ['Biography', 'Crime', 'Drama']\nfirst_page_summary: An outcast New York City cop is charged with bringing down Harlem drug lord Frank Lucas, whose real life inspired this partly biographical film.", metadata={'source': 'data/imdb.csv', 'row': 2433}), Document(page_content='id: tt0102603\ntitle: Oscar\ngenres: [\'Comedy\', \'Crime\']\nfirst_page_summary: A gangster attempts to keep the promise he made to his dying father: that he would give up his life of crime and "go stra

In [17]:
A = c.chat('give me a newer one')
print(A)

<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

[Document(page_content="id: tt27555922\ntitle: Goodfellas 2\ngenres: ['Short', 'Comedy']\nfirst_page_summary: Two gangsters meet and fall madly in love, and have a little bit of gay romance but their Italian brothers won't let that one slide so they all hit the gritty hop on the Fortnite and win som... Read all", metadata={'source': 'data/imdb.csv', 'row': 3951}), Document(page_content="id: tt0765429\ntitle: American Gangster\ngenres: ['Biography', 'Crime', 'Drama']\nfirst_page_summary: An outcast New York City cop is charged with bringing down Harlem drug lord Frank Lucas, whose real life inspired this partly biographical film.", metadata={'source': 'data/imdb.csv', 'row': 2433}), Document(page_content="id: tt0099685\ntitle: Goodfellas\ngenres: ['Biography', 'Crime', 'Drama']\nfirst_page_summary: The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob